In [1]:
import os
import random
import time
import json
import warnings

import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.base import BaseEstimator
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.metrics import fbeta_score
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from xgboost import XGBClassifier 

In [2]:
import sys
sys.path += ['/kaggle/input/iterstrat', '/kaggle/input/tabnet']

from ml_stratifiers import MultilabelStratifiedKFold
from tabnet.stacked_tabnet import StackedTabNetClassifier

In [3]:
def set_random_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
set_random_seeds(43)

In [4]:
X_full = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
Y_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
Y_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')

X_test_full = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sig_id = X_test_full['sig_id'].values.reshape((-1, 1))

In [5]:
cat_cols = ['cp_type', 'cp_time', 'cp_dose']
numerical_cols = [c for c in X_full.columns if c not in ['sig_id'] + cat_cols]
label_cols = [c for c in Y_scored.columns if c != 'sig_id']

num_features = X_full.shape[1] - 1
num_labels = Y_scored.shape[1] - 1
num_nonscored_labels = Y_nonscored.shape[1] - 1

In [6]:
def logloss(Y_true, Y_preds, label_smoothing=0, remove_unpredicted=False):
    if remove_unpredicted:
        predicted = (Y_preds.sum(axis=1) != 0)
        Y_true = Y_true.loc[predicted, :]
        Y_preds = Y_preds.loc[predicted, :]
    Y_true = Y_true.astype(np.float32)
    return tf.reduce_mean(keras.losses.binary_crossentropy(Y_true, Y_preds, label_smoothing=label_smoothing)).numpy()


def metrics(Y_true, Y_preds):
    pr = tf.keras.metrics.Precision()
    pr.update_state(Y_true, Y_preds)
    re = tf.keras.metrics.Recall()
    re.update_state(Y_true, Y_preds)

    pr = pr.result().numpy()
    re = re.result().numpy()
    f1 = 2 * (pr * re) / (pr + re)
    
    auc = tf.keras.metrics.AUC(curve='PR', multi_label=True)
    auc.update_state(Y_true, Y_preds)
    return pr, re, f1, auc.result().numpy()

In [7]:
def add_outputs(Y, Y_preds, X=None, idx_list=None, col=None):
    if X is not None:
        if col is None:
            Y.loc[X['cp_type'] == 'trt_cp', :] += Y_preds
        else:
            Y.loc[X['cp_type'] == 'trt_cp', Y.columns[col]] += Y_preds
    elif idx_list is not None:
        if col is None:
            Y.iloc[idx_list, :] += Y_preds
        else:
            Y.iloc[idx_list, col] += Y_preds
    else:
        raise Exception("Must provide X or idx_list")

In [8]:
def balance_class_weights(Y_true, class_weights=(0.7, 1.6)):
    n_labels = Y_true.shape[1]
    weights = np.empty([n_labels, 2])
    for i in range(n_labels):
        if class_weights == 'balanced':
            total = len(Y_true[:, i])
            pos = np.sum(Y_true[:, i])
            neg = total - pos
            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0
            weights[i] = [weight_for_0, weight_for_1]
        else:
            weights[i] = np.array(class_weights)
    return weights


def weighted_binary_crossentropy(weights, label_smoothing=0):
    def weighted_loss(Y_true, Y_pred):
        bce = tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing)
        return K.mean(
            (weights[:, 0]**(1 - Y_true)) * (weights[:, 1]**(Y_true)) * bce(Y_true, Y_pred),
            axis=-1
        )
    return weighted_loss

In [9]:
def clear_session():
    curr_session = tf.compat.v1.get_default_session()
    if curr_session is not None:
        curr_session.close()
    K.clear_session()
    
    s = tf.compat.v1.InteractiveSession()
    tf.compat.v1.keras.backend.set_session(s)

# Preprocessing

In [10]:
def preprocess(df, targets=None):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df = df.drop(columns='sig_id')
    
    if targets is not None:
        targets = targets.copy()
        targets = targets.drop(columns='sig_id')
        targets = targets[df['cp_type'] == 0].reset_index(drop=True)

    df = df[df['cp_type'] == 0].reset_index(drop=True)
    
    if targets is None:
        return df
    return df, targets

In [11]:
def complete_train_labels(Y, train_idx, test_idx):
    """ Make sure the train set has at least 1 positive class for every label """
    train_idx, test_idx = list(train_idx), list(test_idx)
    Y_train = Y.iloc[train_idx, :]
    
    train_pos_cnts = Y_train.sum(axis=0)
    missing_labels = train_pos_cnts[train_pos_cnts == 0].index
    removed_from_test = []
    
    for l in missing_labels:
        for i in test_idx:
            if Y[l].iloc[i] == 1:
                train_idx.append(i)
                removed_from_test.append(i)
                break
    test_idx = [i for i in test_idx if i not in removed_from_test]
    return train_idx, test_idx

In [12]:
X_train, Y_train = preprocess(X_full, Y_scored)
X_test = preprocess(X_test_full)
Y_train_nonscored = Y_nonscored.drop(columns='sig_id')[X_full['cp_type'] == 'trt_cp'].reset_index(drop=True)
initial_bias = -np.log(Y_train.mean(axis=0).values)
nonscored_initial_bias = -np.log(Y_train_nonscored.mean(axis=0).values + 1e-6)

In [13]:
def multilabel_split_by_drugs(scored, n_folds, seed):
    scored = scored.copy().merge(drug, on='sig_id', how='left') 

    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[vc <= 18].index
    vc2 = vc.loc[vc > 18].index

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}
    dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    tmp = scored.groupby('drug_id')[label_cols].mean().loc[vc1]
    for fold, (train_idx, test_idx) in enumerate(skf.split(tmp, tmp[label_cols])):
        dd = {k:fold for k in tmp.index[test_idx].values}
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
    for fold, (train_idx, test_idx) in enumerate(skf.split(tmp, tmp[label_cols])):
        dd = {k:fold for k in tmp.sig_id[test_idx].values}
        dct2.update(dd)

    # ASSIGN FOLDS
    scored['fold'] = scored.drug_id.map(dct1)
    scored.loc[scored.fold.isna(), 'fold'] = scored.loc[scored.fold.isna(), 'sig_id'].map(dct2)
    scored.fold = scored.fold.astype('int8')
    
    test_idx = [np.where(scored['fold'] == fold)[0].tolist() for fold in range(n_folds)]
    train_idx = [np.where(scored['fold'] != fold)[0].tolist() for fold in range(n_folds)]
    return zip(train_idx, test_idx)

In [14]:
def select_features(X):
    return X.iloc[:, [
        1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874
    ]].values

In [15]:
gene_cols = [c for c in X_train.columns if c.startswith('g-')]
cell_cols = [c for c in X_train.columns if c.startswith('c-')]

In [16]:
import joblib

qtransform = joblib.load('/kaggle/input/qtransform/qtransform.joblib')

X_train_selected = select_features(X_train)
X_test_selected = select_features(X_test)
X_train_qtrans = np.concatenate([X_train_selected[:, :2], qtransform.transform(X_train_selected[:, 2:])], axis=1)
X_test_qtrans =  np.concatenate([X_test_selected[:, :2], qtransform.transform(X_test_selected[:, 2:])], axis=1)

pca_qtrans_2 = joblib.load('/kaggle/input/qtransform/pca_qtrans.joblib')

In [17]:
std_scaler_2 = joblib.load('/kaggle/input/qtransform/std_scaler_2.joblib')
gene_pca_2 = joblib.load('/kaggle/input/qtransform/gene_pca_2.joblib')
cell_pca_2 = joblib.load('/kaggle/input/qtransform/cell_pca_2.joblib')

In [18]:
from sklearn.cluster import KMeans

kmeans_g = joblib.load('/kaggle/input/qtransform/kmeans_g.joblib')
kmeans_c = joblib.load('/kaggle/input/qtransform/kmeans_c.joblib')

# NN

In [19]:
LOAD_PRETRAINED_NN = True
REUSE_SPLITS_NN = True

In [20]:
def preprocess_nn(X):
    X_pca = pca_qtrans_2.transform(X)
    return np.concatenate([X, X_pca], axis=1)

In [21]:
X_train_nn = preprocess_nn(X_train_qtrans)
X_test_nn = preprocess_nn(X_test_qtrans)

In [22]:
!mkdir model_nn

In [23]:
class MultiLabelNN(BaseEstimator):

    def __init__(self,
                 n_input_features=num_features,
                 n_output_labels=num_labels,
                 n_hidden_units=2048):
        self.n_input_features = n_input_features
        self.n_output_labels = n_output_labels
        self.n_hidden_units = n_hidden_units
        self._model_nn = None
        
    def _create_nn_model(self, loss_fn=None, loss_fn_aux=None):
        output_bias = keras.initializers.Constant(initial_bias)
        aux_output_bias = keras.initializers.Constant(nonscored_initial_bias)
        
        inp = keras.layers.Input(self.n_input_features)
        gauss_noise = keras.layers.GaussianNoise(0.001)(inp)
        norm_1 = keras.layers.BatchNormalization()(gauss_noise)

        dense_1 = tfa.layers.WeightNormalization(keras.layers.Dense(
            self.n_hidden_units // 2,
            activation='elu',
            kernel_initializer='he_normal',
            kernel_regularizer=regularizers.l2(1e-5)
        ))(norm_1)
        norm_2 = keras.layers.BatchNormalization()(dense_1)
        dropout_1 = keras.layers.Dropout(0.4)(norm_2)
        
        dense_2 = tfa.layers.WeightNormalization(keras.layers.Dense(
            self.n_hidden_units,
            activation='elu',
            kernel_initializer='he_normal',
            kernel_regularizer=regularizers.l2(1e-5)
        ))(dropout_1)
        norm_3 = keras.layers.BatchNormalization()(dense_2)
        dropout_2 = keras.layers.Dropout(0.4)(norm_3)
        
        dropout_3 = keras.layers.Dropout(0.5)(norm_2)
        dense_3 = tfa.layers.WeightNormalization(keras.layers.Dense(
            self.n_hidden_units // 2,
            activation='relu',
            kernel_regularizer=regularizers.l2(1e-5)
        ))(dropout_3)
        norm_4 = keras.layers.BatchNormalization()(dense_3)
        dropout_4 = keras.layers.Dropout(0.4)(norm_4)

        output_1 = tfa.layers.WeightNormalization(keras.layers.Dense(
            self.n_output_labels,
            activation='sigmoid',
            bias_initializer=output_bias
        ), name='main')(dropout_4)
        output_2 = tfa.layers.WeightNormalization(keras.layers.Dense(
            Y_train_nonscored.shape[1],
            activation='sigmoid',
            bias_initializer=aux_output_bias
        ), name='aux')(dropout_2)
        
        model_nn = keras.models.Model(inputs=inp, outputs=[output_1, output_2])
        model_nn.compile(
            loss=[
                keras.losses.BinaryCrossentropy(label_smoothing=2e-4) if loss_fn is None else loss_fn,
                keras.losses.BinaryCrossentropy(label_smoothing=2e-4) if loss_fn_aux is None else loss_fn_aux
            ],
            optimizer=keras.optimizers.Nadam(learning_rate=0.01),
            metrics=['binary_crossentropy'],
            loss_weights=[0.9, 0.1]
        )
        return model_nn
        
    def fit(self, X, Y, X_valid, Y_valid, Y_aux, Y_aux_valid, max_epochs=100, save_weights=True):
        Y = tf.cast(Y, tf.float32)
        Y_valid = tf.cast(Y_valid, tf.float32)
        Y_aux = tf.cast(Y_aux, tf.float32)
        Y_aux_valid = tf.cast(Y_aux_valid, tf.float32)

        class_weights = balance_class_weights(Y.numpy(), class_weights=(0.7, 2.0))
        loss_fn = weighted_binary_crossentropy(class_weights, label_smoothing=1e-3)
        class_weights_aux = balance_class_weights(Y_aux.numpy(), class_weights=(0.7, 1.4))
        loss_fn_aux = weighted_binary_crossentropy(class_weights_aux, label_smoothing=1e-3)
        self._model_nn = self._create_nn_model(loss_fn=loss_fn, loss_fn_aux=loss_fn_aux)

        learning_rate_cb = keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3, verbose=1, factor=0.1, min_lr=1e-8
        )
        early_stop_cb = keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.0001, patience=10,
            restore_best_weights=False
        )
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
            f'model_nn/weights_{repeat}_{fold}.h5',
            monitor='val_main_binary_crossentropy',
            save_best_only=True,
            save_weights_only=True
        )
        callbacks = [early_stop_cb, learning_rate_cb, checkpoint_cb] if save_weights else [early_stop_cb, learning_rate_cb]

        history = self._model_nn.fit(
            X, [Y, Y_aux],
            batch_size=128,
            epochs=max_epochs,
            validation_data=(X_valid, [Y_valid, Y_aux_valid]),
            callbacks=callbacks,
            shuffle=True,
            verbose=0
        )
        return history
    
    def predict(self, X):
        return self._model_nn.predict(X)
    
    def score(self, X, Y_true):
        Y_preds = self.predict(X)[0]
        return logloss(Y_true, Y_preds)
    
    def save_weights(self, repeat, fold):
        self._model_nn.save_weights('model_nn/weights_{}_{}.h5'.format(repeat, fold))
        
    def load_weights(self, repeat, fold, pretrained=True):
        if self._model_nn is None:
            self._model_nn = self._create_nn_model()
        if pretrained:
            base_path = '/kaggle/input/model-nn/'
        else:
            base_path = './working/'
        self._model_nn.load_weights(base_path + 'model_nn/weights_{}_{}.h5'.format(repeat, fold))

In [24]:
n_repeats = 5
n_folds = 7
kfold_seeds = [74, 1156, 415, 608, 1993] if REUSE_SPLITS_NN else np.random.randint(42, 1337, n_repeats)
kfold_seeds = kfold_seeds[:n_repeats]
assert len(kfold_seeds) == n_repeats
print(kfold_seeds)

[74, 1156, 415, 608, 1993]


In [25]:
val_losses = []
histories = []
Y_train_preds_nn = pd.DataFrame(np.zeros((X_train.shape[0], num_labels)), columns=label_cols)
Y_test_preds_nn = pd.DataFrame(np.zeros((X_test_full.shape[0], num_labels)), columns=label_cols)

set_random_seeds(42)

for repeat, kf_seed in enumerate(kfold_seeds):
    kf = MultilabelStratifiedKFold(n_splits=n_folds, random_state=kf_seed, shuffle=True)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X_train, Y_train)):
        print('** Repeat {}/{}. Fold {}/{}'.format(repeat + 1, n_repeats, fold + 1, n_folds))
        K.clear_session()
        train_idx, test_idx = complete_train_labels(Y_train, train_idx, test_idx)
        train_idx = tf.random.shuffle(train_idx)
        X_train_fold = tf.gather(X_train_nn, train_idx, axis=0)
        Y_train_fold = tf.gather(Y_train, train_idx, axis=0)
        Y_train_aux_fold = tf.gather(Y_train_nonscored, train_idx, axis=0)
        X_valid_fold = tf.gather(X_train_nn, test_idx, axis=0)
        Y_valid_fold = tf.gather(Y_train, test_idx, axis=0)
        Y_valid_aux_fold = tf.gather(Y_train_nonscored, test_idx, axis=0)

        model_nn = MultiLabelNN(n_input_features=X_train_nn.shape[1])
        if LOAD_PRETRAINED_NN:
            model_nn.load_weights(repeat, fold)
            val_loss = model_nn.score(X_valid_fold, Y_valid_fold.numpy())
        else:
            history = model_nn.fit(
                X_train_fold, Y_train_fold,
                X_valid_fold, Y_valid_fold,
                Y_train_aux_fold, Y_valid_aux_fold
            ) 
            histories.append(history)
            val_loss = min(history.history['val_main_binary_crossentropy'])
            model_nn.load_weights(repeat, fold, pretrained=False)

        val_preds = model_nn.predict(X_valid_fold)[0]
        add_outputs(Y_train_preds_nn, val_preds, idx_list=test_idx)

        test_preds = model_nn.predict(X_test_nn)[0]
        add_outputs(Y_test_preds_nn, test_preds, X=X_test_full)

        val_losses.append(val_loss)
        print('Val loss: {}'.format(val_loss))
        
        del model_nn

    print('----------')
    print('Repeat avg val loss: {}'.format(np.mean(val_losses[repeat*n_folds:((repeat + 1)*n_folds)])))
    print('Repeat OOF val loss: {}'.format(logloss(Y_train, Y_train_preds_nn / (repeat + 1), remove_unpredicted=True)))
    print('----------')

Y_train_preds_nn /= n_repeats       
Y_test_preds_nn.loc[X_test_full['cp_type'] == 'trt_cp', :] /= (n_folds * n_repeats)
print('==========')
print('Overall avg val loss: {}'.format(np.mean(val_losses)))
print('Overall OOF val loss: {}'.format(logloss(Y_train, Y_train_preds_nn, remove_unpredicted=True)))

** Repeat 1/5. Fold 1/7
Val loss: 0.015600921586155891
** Repeat 1/5. Fold 2/7
Val loss: 0.015657443553209305
** Repeat 1/5. Fold 3/7
Val loss: 0.015989556908607483
** Repeat 1/5. Fold 4/7
Val loss: 0.015861568972468376
** Repeat 1/5. Fold 5/7
Val loss: 0.01586352474987507
** Repeat 1/5. Fold 6/7
Val loss: 0.01587575115263462
** Repeat 1/5. Fold 7/7
Val loss: 0.015924906358122826
----------
Repeat avg val loss: 0.015824811533093452
Repeat OOF val loss: 0.0158248473212246
----------
** Repeat 2/5. Fold 1/7
Val loss: 0.01592746190726757
** Repeat 2/5. Fold 2/7
Val loss: 0.015704503282904625
** Repeat 2/5. Fold 3/7
Val loss: 0.015848109498620033
** Repeat 2/5. Fold 4/7
Val loss: 0.015885770320892334
** Repeat 2/5. Fold 5/7
Val loss: 0.015708347782492638
** Repeat 2/5. Fold 6/7
Val loss: 0.015785587951540947
** Repeat 2/5. Fold 7/7
Val loss: 0.015963325276970863
----------
Repeat avg val loss: 0.015831872820854187
Repeat OOF val loss: 0.01571310308753488
----------
** Repeat 3/5. Fold 1/7


In [26]:
# !zip -r model_nn.zip model_nn
# !rm -r model_nn

In [27]:
def plot_history(history):
    df = pd.DataFrame({
        'loss': history.history['main_binary_crossentropy'],
        'val_loss': history.history['val_main_binary_crossentropy'],
    })
    print(df['val_loss'].min())
    df = df.loc[3:, :]
    df.plot()
    plt.axvline(x=df['val_loss'].idxmin(), color='r')

In [28]:
# plot_history(histories[0])

# TabNet

In [29]:
LOAD_PRETRAINED_TABNET = True
REUSE_SPLITS_TABNET = True

In [30]:
!mkdir model_tabnet

In [31]:
def preprocess_tabnet(X, X_qtrans):
    X_pca = pca_qtrans_2.transform(X_qtrans)
    g_std = X[gene_cols].std(axis=1).values.reshape(-1, 1)
    g_kurt = X[gene_cols].kurtosis(axis=1).values.reshape(-1, 1)
    g_skew = X[gene_cols].skew(axis=1).values.reshape(-1, 1)
    c_std = X[cell_cols].std(axis=1).values.reshape(-1, 1)
    c_kurt = X[cell_cols].kurtosis(axis=1).values.reshape(-1, 1)
    c_skew = X[cell_cols].skew(axis=1).values.reshape(-1, 1)
    X_std = pd.DataFrame(std_scaler_2.transform(X[numerical_cols]), columns=numerical_cols)
    g_cluster = kmeans_g.predict(X_std[gene_cols]).reshape(-1, 1)
    c_cluster = kmeans_c.predict(X_std[cell_cols]).reshape(-1, 1)
    return np.concatenate([
        X_qtrans, X_pca,
        g_std, g_kurt, g_skew,
        c_std, c_kurt, c_skew,
        g_cluster, c_cluster
    ], axis=1)

In [32]:
X_train_tabnet = preprocess_tabnet(X_train, X_train_qtrans)
X_test_tabnet = preprocess_tabnet(X_test, X_test_qtrans)

In [33]:
class MultiLabelTabNet(BaseEstimator):

    def __init__(self, n_input_features, n_output_labels):
        self.n_input_features = n_input_features
        self.n_output_labels = n_output_labels
        self.estimator_ = None
        
    def _create_model(self, loss_fn=None, output_bias=None):
        if output_bias is not None:
            output_bias = keras.initializers.Constant(output_bias)

        clf = StackedTabNetClassifier(
            feature_columns=None, 
            num_classes=1024,
            num_layers=1,
            feature_dim=2048,
            output_dim=1024,
            num_features=self.n_input_features,
            num_decision_steps=1,
            relaxation_factor=1.5,
            sparsity_coefficient=1e-5,
            num_groups=-1,
            output_activation='relu'
        )
        model = keras.models.Sequential([
            keras.layers.InputLayer(self.n_input_features),
            keras.layers.Dropout(0.5),
            keras.layers.GaussianNoise(0.001),
            clf,
            keras.layers.Dropout(0.7),
            tfa.layers.WeightNormalization(keras.layers.Dense(
                self.n_output_labels,
                activation='sigmoid',
                bias_initializer=output_bias
            ))
        ])
        model.compile(
            loss=keras.losses.BinaryCrossentropy(label_smoothing=2e-3) if loss_fn is None else loss_fn,
            optimizer=tfa.optimizers.AdamW(weight_decay=1e-5),
            metrics=['binary_crossentropy']
        )
        return model
        
    def fit(self, X, Y, X_valid, Y_valid, repeat, fold, max_epochs=200):
        Y = tf.cast(Y, tf.float32)
        Y_valid = tf.cast(Y_valid, tf.float32)
        class_weights = balance_class_weights(Y.numpy(), class_weights=(0.5, 15.0))
        loss_fn = weighted_binary_crossentropy(class_weights, label_smoothing=2e-3)
        self.estimator_ = self._create_model(loss_fn=loss_fn, output_bias=initial_bias)

        learning_rate_cb = keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', min_delta=1e-5,
            patience=3, verbose=1, factor=0.1, min_lr=1e-7
        )
        early_stop_cb = keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=1e-5, patience=10, restore_best_weights=False
        )
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
            f'model_tabnet/weights_{repeat}_{fold}.h5',
            monitor='val_binary_crossentropy',
            save_best_only=True,
            save_weights_only=True
        )

        history = self.estimator_.fit(
            X, Y,
            batch_size=128,
            epochs=max_epochs,
            validation_data=(X_valid, Y_valid),
            callbacks=[early_stop_cb, learning_rate_cb, checkpoint_cb],
            shuffle=True,
            verbose=0
        )
        return history
    
    def predict(self, X):
        return self.estimator_.predict(X)
    
    def score(self, X, Y_true):
        Y_preds = self.predict(X)
        return logloss(Y_true, Y_preds)
    
    def save_weights(self, repeat, fold):
        self.estimator_.save_weights('model_tabnet/weights_{}_{}.h5'.format(repeat, fold))
        
    def load_weights(self, repeat, fold, pretrained=True):
        if self.estimator_ is None:
            self.estimator_ = self._create_model()
        if pretrained:
            base_path = '/kaggle/input/model-tabnet/'
        else:
            base_path = './'
        self.estimator_.load_weights(base_path + f'model_tabnet/weights_{repeat}_{fold}.h5')

In [34]:
n_repeats = 5
n_folds = 7
kfold_seeds = [1217, 231, 902, 3893, 111] if REUSE_SPLITS_TABNET else np.random.randint(42, 1337, n_repeats)
kfold_seeds = kfold_seeds[:n_repeats]
assert len(kfold_seeds) == n_repeats
print(kfold_seeds)

[1217, 231, 902, 3893, 111]


In [35]:
val_losses = []
histories = []
Y_train_preds_tabnet = pd.DataFrame(np.zeros((X_train.shape[0], num_labels)), columns=label_cols)
Y_test_preds_tabnet = pd.DataFrame(np.zeros((X_test_full.shape[0], num_labels)), columns=label_cols)

set_random_seeds(888)

for repeat, kf_seed in enumerate(kfold_seeds):
    kf = MultilabelStratifiedKFold(n_splits=n_folds, random_state=kf_seed, shuffle=True)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X_train_tabnet, Y_train)):
        print('** Repeat {}/{}. Fold {}/{}'.format(repeat + 1, n_repeats, fold + 1, n_folds))
        K.clear_session()
        train_idx, test_idx = complete_train_labels(Y_train, train_idx, test_idx)
        train_idx = tf.random.shuffle(train_idx)
        X_train_fold = tf.gather(X_train_tabnet, train_idx, axis=0)
        Y_train_fold = tf.gather(Y_train, train_idx, axis=0)
        X_valid_fold = tf.gather(X_train_tabnet, test_idx, axis=0)
        Y_valid_fold = tf.gather(Y_train, test_idx, axis=0)

        model_tabnet = MultiLabelTabNet(
            n_input_features=X_train_tabnet.shape[1],
            n_output_labels=Y_train.shape[1]
        )
        if LOAD_PRETRAINED_TABNET:
            model_tabnet.load_weights(repeat, fold)
            val_loss = model_tabnet.score(X_valid_fold, Y_valid_fold.numpy())
        else:
            history = model_tabnet.fit(
                X_train_fold, Y_train_fold,
                X_valid_fold, Y_valid_fold,
                repeat, fold
            )
            histories.append(history)
            val_loss = min(history.history['val_binary_crossentropy'])
            model_tabnet.load_weights(repeat, fold, pretrained=False)

        val_preds = model_tabnet.predict(X_valid_fold)
        add_outputs(Y_train_preds_tabnet, val_preds, idx_list=test_idx)

        test_preds = model_tabnet.predict(X_test_tabnet)
        add_outputs(Y_test_preds_tabnet, test_preds, X=X_test_full)

        val_losses.append(val_loss)
        print('Val loss: {}'.format(val_loss))
        
        del model_tabnet

    print('----------')
    print('Repeat avg val loss: {}'.format(np.mean(val_losses[repeat*n_folds:((repeat + 1)*n_folds)])))
    print('Repeat OOF val loss: {}'.format(logloss(Y_train, Y_train_preds_tabnet / (repeat + 1), remove_unpredicted=True)))
    print('----------')

Y_train_preds_tabnet /= n_repeats
Y_test_preds_tabnet.loc[X_test_full['cp_type'] == 'trt_cp', :] /= (n_folds * n_repeats)
print('==========')
print('Overall avg val loss: {}'.format(np.mean(val_losses)))
print('Overall OOF val loss: {}'.format(logloss(Y_train, Y_train_preds_tabnet, remove_unpredicted=True)))

** Repeat 1/5. Fold 1/7
Val loss: 0.015734296292066574
** Repeat 1/5. Fold 2/7
Val loss: 0.015842722728848457
** Repeat 1/5. Fold 3/7
Val loss: 0.016170810908079147
** Repeat 1/5. Fold 4/7
Val loss: 0.015860486775636673
** Repeat 1/5. Fold 5/7
Val loss: 0.01592175103724003
** Repeat 1/5. Fold 6/7
Val loss: 0.01600259356200695
** Repeat 1/5. Fold 7/7
Val loss: 0.016120461747050285
----------
Repeat avg val loss: 0.01595044694840908
Repeat OOF val loss: 0.01595043474063799
----------
** Repeat 2/5. Fold 1/7
Val loss: 0.015831606462597847
** Repeat 2/5. Fold 2/7
Val loss: 0.015790700912475586
** Repeat 2/5. Fold 3/7
Val loss: 0.016097530722618103
** Repeat 2/5. Fold 4/7
Val loss: 0.015869125723838806
** Repeat 2/5. Fold 5/7
Val loss: 0.016081590205430984
** Repeat 2/5. Fold 6/7
Val loss: 0.016186177730560303
** Repeat 2/5. Fold 7/7
Val loss: 0.015976887196302414
----------
Repeat avg val loss: 0.015976231545209885
Repeat OOF val loss: 0.015729820695384318
----------
** Repeat 3/5. Fold 1/

In [36]:
# !zip -r model_tabnet.zip model_tabnet
# !rm -r model_tabnet

In [37]:
def plot_history(history):
    df = pd.DataFrame({
        'loss': history.history['loss'],
        'val_loss': history.history['val_loss'],
    })
    print(df['val_loss'].min())
    df = df.loc[1:, :]
    df.plot()
    plt.axvline(x=df['val_loss'].idxmin(), color='r')

In [38]:
# plot_history(histories[5])

# ResNet

In [39]:
LOAD_PRETRAINED_RESNET = True
REUSE_SPLITS_RESNET = True

In [40]:
def preprocess_resnet(X):
    X_scaled = pd.DataFrame(std_scaler_2.transform(X[numerical_cols]), columns=numerical_cols)
    X_genes = gene_pca_2.transform(X_scaled[gene_cols])
    X_cells = cell_pca_2.transform(X_scaled[cell_cols])
    return select_features(X), np.concatenate([X_genes, X_cells], axis=1)

In [41]:
X_train_resnet_1, X_train_resnet_2 = preprocess_resnet(X_train)
X_test_resnet_1, X_test_resnet_2 = preprocess_resnet(X_test)

In [42]:
!mkdir model_resnet

In [43]:
class MultiLabelResNet(BaseEstimator):

    def __init__(self, n_input_1, n_input_2, n_output_labels):
        self.n_input_1 = n_input_1
        self.n_input_2 = n_input_2
        self.n_output_labels = n_output_labels
        self.estimator_ = None
        
    def _create_model(self, initial_bias, loss_fn=None):
        output_bias = keras.initializers.Constant(initial_bias)
        
        inp_1 = keras.layers.Input(self.n_input_1, name='raw_inp')
        inp_2 = keras.layers.Input(self.n_input_2, name='pca_inp')
        gauss_2 = keras.layers.GaussianNoise(1e-4)(inp_2)

        head_1 = keras.models.Sequential([
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.2),
            tfa.layers.WeightNormalization(keras.layers.Dense(512, activation='elu')),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.5),
            tfa.layers.WeightNormalization(keras.layers.Dense(256, activation='elu'))
        ], name='head1') 

        seq_1 = head_1(inp_1)
        seq_1_inp_concat = keras.layers.Concatenate()([gauss_2, seq_1])

        head_2 = keras.models.Sequential([
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.4),
            tfa.layers.WeightNormalization(keras.layers.Dense(512, "relu")),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.5),
            tfa.layers.WeightNormalization(keras.layers.Dense(512, "elu")),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.5),
            tfa.layers.WeightNormalization(keras.layers.Dense(256, "relu")),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.1),
            tfa.layers.WeightNormalization(keras.layers.Dense(256, "elu"))
        ], name='head2')

        seq_2 = head_2(seq_1_inp_concat)
        seq_1_seq_2_avg = keras.layers.Average()([seq_1, seq_2]) 

        head_3 = keras.models.Sequential([
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.1),
            tfa.layers.WeightNormalization(keras.layers.Dense(256, activation='relu')),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.1),
            tfa.layers.WeightNormalization(keras.layers.Dense(206, activation='relu')),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.1),
            tfa.layers.WeightNormalization(keras.layers.Dense(self.n_output_labels, bias_initializer=output_bias, activation='sigmoid'))
        ], name='head3')

        output = head_3(seq_1_seq_2_avg)
        
        model = keras.models.Model(inputs=[inp_1, inp_2], outputs=output)
        model.compile(
            loss=keras.losses.BinaryCrossentropy(label_smoothing=2e-3) if loss_fn is None else loss_fn,
            optimizer=tfa.optimizers.AdamW(learning_rate=0.02, weight_decay=1e-5),
            metrics=['binary_crossentropy'],
        )
        return model

    def fit(self, X, Y, X_valid, Y_valid, repeat, fold, max_epochs=100, save_weights=True, initial_bias=initial_bias, transfer_model=None):
        Y = tf.cast(Y, tf.float32)
        Y_valid = tf.cast(Y_valid, tf.float32)

        class_weights = balance_class_weights(Y.numpy(), class_weights=(0.5, 2.0))
        loss_fn = weighted_binary_crossentropy(class_weights, label_smoothing=2e-3)
        self.estimator_ = self._create_model(initial_bias, loss_fn=loss_fn)
        if transfer_model is not None:
            self.copy_weights(transfer_model)

        learning_rate_cb = keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3, verbose=1, factor=0.1, min_lr=1e-8
        )
        early_stop_cb = keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.0001, patience=15,
            restore_best_weights=False
        )
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
            f'model_resnet/weights_{repeat}_{fold}.h5',
            monitor='val_binary_crossentropy',
            save_best_only=True,
            save_weights_only=True
        )
        callbacks = [early_stop_cb, learning_rate_cb, checkpoint_cb] if save_weights else [early_stop_cb, learning_rate_cb]

        history = self.estimator_.fit(
            X, Y,
            batch_size=128,
            epochs=max_epochs,
            validation_data=(X_valid, Y_valid),
            callbacks=callbacks,
            shuffle=True,
            verbose=0
        )
        return history
    
    def predict(self, X):
        return self.estimator_.predict(X)
    
    def score(self, X, Y_true):
        Y_preds = self.predict(X)
        return logloss(Y_true, Y_preds)
    
    def save_weights(self, repeat, fold):
        self.estimator_.save_weights('model_resnet/weights_{}_{}.h5'.format(repeat, fold))
        
    def load_weights(self, repeat, fold, pretrained=True):
        if self.estimator_ is None:
            self.estimator_ = self._create_model(initial_bias)
        if pretrained:
            base_path = '/kaggle/input/model-resnet/'
        else:
            base_path = './'
        self.estimator_.load_weights(base_path + 'model_resnet/weights_{}_{}.h5'.format(repeat, fold))
        
    def copy_weights(self, from_model):
        for to_layer, from_layer in zip(self.estimator_.layers[:-1], from_model.estimator_.layers[:-1]):
            to_layer.set_weights(from_layer.get_weights())
        for to_layer, from_layer in zip(self.estimator_.layers[-1].layers[:-1], from_model.estimator_.layers[-1].layers[:-1]):
            to_layer.set_weights(from_layer.get_weights())

In [44]:
n_repeats = 7
n_folds = 7
kfold_seeds = [188, 526, 1045, 453, 200, 8684, 87352] if REUSE_SPLITS_RESNET else np.random.randint(42, 1337, n_repeats)
kfold_seeds = kfold_seeds[:n_repeats]
assert len(kfold_seeds) == n_repeats
print(kfold_seeds)

[188, 526, 1045, 453, 200, 8684, 87352]


In [45]:
val_losses = []
histories = []
Y_train_preds_resnet = pd.DataFrame(np.zeros((X_train.shape[0], num_labels)), columns=label_cols)
Y_test_preds_resnet = pd.DataFrame(np.zeros((X_test_full.shape[0], num_labels)), columns=label_cols)

set_random_seeds(123)

for repeat, kf_seed in enumerate(kfold_seeds):
    kf = MultilabelStratifiedKFold(n_splits=n_folds, random_state=kf_seed, shuffle=True)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X_train, Y_train)):
        print('** Repeat {}/{}. Fold {}/{}'.format(repeat + 1, n_repeats, fold + 1, n_folds))
        K.clear_session()
        train_idx, test_idx = complete_train_labels(Y_train, train_idx, test_idx)
        train_idx = tf.random.shuffle(train_idx)
        X_train_1_fold = tf.gather(X_train_resnet_1, train_idx, axis=0)
        X_train_2_fold = tf.gather(X_train_resnet_2, train_idx, axis=0)
        Y_train_fold = tf.gather(Y_train, train_idx, axis=0)
        X_valid_1_fold = tf.gather(X_train_resnet_1, test_idx, axis=0)
        X_valid_2_fold = tf.gather(X_train_resnet_2, test_idx, axis=0)
        Y_valid_fold = tf.gather(Y_train, test_idx, axis=0)
        Y_nonscored_train_fold = tf.gather(Y_train_nonscored, train_idx, axis=0)
        Y_nonscored_valid_fold = tf.gather(Y_train_nonscored, test_idx, axis=0)

        model_resnet = MultiLabelResNet(
            n_input_1=X_train_resnet_1.shape[1],
            n_input_2=X_train_resnet_2.shape[1],
            n_output_labels=Y_train.shape[1]
        )
        if LOAD_PRETRAINED_RESNET:
            model_resnet.load_weights(repeat, fold)
            val_loss = model_resnet.score([X_valid_1_fold, X_valid_2_fold], Y_valid_fold.numpy())
        else:
            print('Pretrain model')
            transfer_model = MultiLabelResNet(
                n_input_1=X_train_resnet_1.shape[1],
                n_input_2=X_train_resnet_2.shape[1],
                n_output_labels=Y_train_nonscored.shape[1]
            )
            transfer_model.fit(
                [X_train_1_fold, X_train_2_fold], Y_nonscored_train_fold,
                [X_valid_1_fold, X_valid_2_fold], Y_nonscored_valid_fold,
                repeat, fold,
                max_epochs=10,
                save_weights=False,
                initial_bias=nonscored_initial_bias
            )

            print('Train model')
            history = model_resnet.fit(
                [X_train_1_fold, X_train_2_fold], Y_train_fold,
                [X_valid_1_fold, X_valid_2_fold], Y_valid_fold,
                repeat, fold,
                transfer_model=transfer_model
            ) 
            histories.append(history)
            val_loss = min(history.history['val_binary_crossentropy'])
            model_resnet.load_weights(repeat, fold, pretrained=False)
            del transfer_model

        val_preds = model_resnet.predict([X_valid_1_fold, X_valid_2_fold])
        add_outputs(Y_train_preds_resnet, val_preds, idx_list=test_idx)

        test_preds = model_resnet.predict([X_test_resnet_1, X_test_resnet_2])
        add_outputs(Y_test_preds_resnet, test_preds, X=X_test_full)

        val_losses.append(val_loss)
        print('Val loss: {}'.format(val_loss))

        del model_resnet
    
    print('----------')
    print('Repeat avg val loss: {}'.format(np.mean(val_losses[repeat*n_folds:((repeat + 1)*n_folds)])))
    print('Repeat OOF val loss: {}'.format(logloss(Y_train, Y_train_preds_resnet / (repeat + 1), remove_unpredicted=True)))
    print('----------')

Y_train_preds_resnet /= n_repeats       
Y_test_preds_resnet.loc[X_test_full['cp_type'] == 'trt_cp', :] /= (n_folds * n_repeats)
print('==========')
print('Overall avg val loss: {}'.format(np.mean(val_losses)))
print('Overall OOF val loss: {}'.format(logloss(Y_train, Y_train_preds_resnet, remove_unpredicted=True)))

** Repeat 1/7. Fold 1/7
Val loss: 0.01573297567665577
** Repeat 1/7. Fold 2/7
Val loss: 0.016409706324338913
** Repeat 1/7. Fold 3/7
Val loss: 0.01591956429183483
** Repeat 1/7. Fold 4/7
Val loss: 0.015869781374931335
** Repeat 1/7. Fold 5/7
Val loss: 0.015978172421455383
** Repeat 1/7. Fold 6/7
Val loss: 0.015889571979641914
** Repeat 1/7. Fold 7/7
Val loss: 0.016222050413489342
----------
Repeat avg val loss: 0.016003118827939034
Repeat OOF val loss: 0.01600313718700611
----------
** Repeat 2/7. Fold 1/7
Val loss: 0.016249777749180794
** Repeat 2/7. Fold 2/7
Val loss: 0.01603727973997593
** Repeat 2/7. Fold 3/7
Val loss: 0.015846941620111465
** Repeat 2/7. Fold 4/7
Val loss: 0.016024982556700706
** Repeat 2/7. Fold 5/7
Val loss: 0.01614505425095558
** Repeat 2/7. Fold 6/7
Val loss: 0.01601618528366089
** Repeat 2/7. Fold 7/7
Val loss: 0.015901396051049232
----------
Repeat avg val loss: 0.0160316601395607
Repeat OOF val loss: 0.015883193546596
----------
** Repeat 3/7. Fold 1/7
Val l

In [46]:
# !zip -r model_resnet.zip model_resnet
# !rm -r model_resnet

In [47]:
def plot_history(history):
    df = pd.DataFrame({
        'loss': history.history['binary_crossentropy'],
        'val_loss': history.history['val_binary_crossentropy'],
    })
    print(df['val_loss'].min())
    df = df.loc[1:, :]
    df.plot()
    plt.axvline(x=df['val_loss'].idxmin(), color='r')

In [48]:
# plot_history(histories[6])

# Blender

In [49]:
# class_weights = balance_class_weights(Y_train.values, class_weights=(0.5, 10.0))
# loss_fn = weighted_binary_crossentropy(class_weights, label_smoothing=2e-3)

# def objective_func(Y_true, Y_preds_list, weights):
#     blended_preds = sum([w*Y_preds for w, Y_preds in zip(weights, Y_preds_list)])
#     penalty = 1e-3*(sum(weights) - 1)**2
#     if min(weights) < 0:
#         penalty += -10*min(weights)
#     return tf.reduce_mean(loss_fn(Y_true, blended_preds)) + penalty

In [50]:
# init_weights = [0.22910333, 0.41949411, 0.35140255]
# print(sum(init_weights))
# assert abs(sum(init_weights) - 1) < 1e-6
# preds_list = [Y_train_preds_nn, Y_train_preds_tabnet, Y_train_preds_resnet]
# val_blended_preds = pd.DataFrame(np.zeros((X_train.shape[0], num_labels)), columns=label_cols)

# losses = []
# val_losses = []
# all_losses = []
# all_weight_vars = []
# num_epochs = 300

# kf = MultilabelStratifiedKFold(n_splits=5, random_state=34, shuffle=True)
# for fold, (train_idx, test_idx) in enumerate(kf.split(np.zeros(X_train.shape), Y_train)):
#     weight_vars = [tf.Variable(w, dtype=tf.float32) for w in init_weights]
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

#     train_idx, test_idx = complete_train_labels(Y_train, train_idx, test_idx)
#     tf_Y_train = tf.constant(Y_train.iloc[train_idx, :], dtype=tf.float32)
#     tf_preds_list = [tf.constant(y.iloc[train_idx, :], dtype=tf.float32) for y in preds_list]
#     tf_Y_valid = tf.constant(Y_train.iloc[test_idx, :], dtype=tf.float32)
#     tf_preds_list_valid = [tf.constant(y.iloc[test_idx, :], dtype=tf.float32) for y in preds_list]
#     fold_losses = []
#     fold_val_losses = []

#     for epoch in range(num_epochs):
#         with tf.GradientTape() as tape:
#             loss_value = objective_func(tf_Y_train, tf_preds_list, weight_vars)
#         grads = tape.gradient(loss_value, weight_vars)
#         optimizer.apply_gradients(zip(grads, weight_vars))
#         print(fold, epoch, loss_value.numpy())
#         fold_losses.append(loss_value.numpy())
#         fold_val_losses.append(objective_func(tf_Y_valid, tf_preds_list_valid, weight_vars).numpy())

#     all_losses.append([fold_losses, fold_val_losses])

#     losses.append(loss_value.numpy())
#     val_loss = objective_func(tf_Y_valid, tf_preds_list_valid, weight_vars).numpy()
#     val_losses.append(val_loss)
# #     print(fold, loss_value.numpy(), val_loss)
    
#     optimized_weights = np.array([v.numpy() for v in weight_vars], dtype=np.float64)
#     optimized_weights /= np.sum(optimized_weights)
#     all_weight_vars.append(optimized_weights)
#     print(optimized_weights)
    
#     val_preds = sum([w*Y_preds for w, Y_preds in zip(optimized_weights, preds_list)])
#     add_outputs(val_blended_preds, val_preds, idx_list=test_idx)
    
# print(np.mean(losses), np.mean(val_losses), logloss(Y_train, val_blended_preds), np.array(all_weight_vars).mean(axis=0))

In [51]:
# plt_fold = 2
# plt.plot(all_losses[plt_fold][0])
# plt.plot(all_losses[plt_fold][1])
# plt.plot(all_losses)

In [52]:
weights = [0.22, 0.43, 0.35]
print(sum(weights))
assert(abs(sum(weights) - 1) < 1e-6)

all_train_preds = (
    Y_train_preds_nn * weights[0] +
    Y_train_preds_tabnet * weights[1] +
    Y_train_preds_resnet * weights[2]
)
print(logloss(Y_train, all_train_preds, remove_unpredicted=True))

1.0
0.015436135491819642


In [53]:
# all_train_preds.to_csv('train_preds.csv', index=False)

# Submission

In [54]:
final_preds = (
    Y_test_preds_nn * weights[0] +
    Y_test_preds_tabnet * weights[1] +
    Y_test_preds_resnet * weights[2]
)
final_preds.loc[X_test_full['cp_type'] != 'trt_cp', :] = 0

In [55]:
final_preds = np.concatenate([sig_id, final_preds], axis=1)
final_preds = pd.DataFrame(final_preds, columns=Y_scored.columns)
final_preds.to_csv('submission.csv', index=False)